## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,46.11,81,99,21.45,light rain
1,1,Ribeira Grande,PT,38.5167,-28.7000,59.83,75,14,14.32,few clouds
2,2,Vardo,NO,70.3705,31.1107,22.51,80,56,8.79,broken clouds
3,3,Qasigiannguit,GL,68.8193,-51.1922,-0.44,93,100,12.95,overcast clouds
4,4,Ossora,RU,59.2353,163.0719,19.96,94,100,13.98,snow


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input('What is the minimum temperature you would like for your trip?'))
max_temp=float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df=city_data_df.loc[(city_data_df['Max Temp']<=max_temp)&(city_data_df['Max Temp']>=min_temp)]
preferred_city_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Mataura,NZ,-46.1927,168.8643,75.47,50,65,1.88,broken clouds
12,12,Carnarvon,AU,-24.8667,113.6333,81.54,61,1,19.13,clear sky
13,13,Souillac,MU,-20.5167,57.5167,79.36,78,75,14.97,broken clouds
16,16,Rikitea,PF,-23.1203,-134.9692,78.01,67,66,4.72,broken clouds
28,28,Sao Filipe,CV,14.8961,-24.4956,75.04,69,100,5.35,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City_ID                170
City                   170
Country                170
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Current Description    170
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_city_df.dropna()
clean_df.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Mataura,NZ,-46.1927,168.8643,75.47,50,65,1.88,broken clouds
12,12,Carnarvon,AU,-24.8667,113.6333,81.54,61,1,19.13,clear sky
13,13,Souillac,MU,-20.5167,57.5167,79.36,78,75,14.97,broken clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Mataura,NZ,75.47,broken clouds,-46.1927,168.8643,
12,Carnarvon,AU,81.54,clear sky,-24.8667,113.6333,
13,Souillac,MU,79.36,broken clouds,-20.5167,57.5167,
16,Rikitea,PF,78.01,broken clouds,-23.1203,-134.9692,
28,Sao Filipe,CV,75.04,overcast clouds,14.8961,-24.4956,
30,Sampit,ID,76.39,overcast clouds,-2.5333,112.9500,
38,Atuona,PF,78.46,few clouds,-9.8000,-139.0333,
40,Kahului,US,76.96,broken clouds,20.8947,-156.4700,
47,Saint-Philippe,RE,78.39,overcast clouds,-21.3585,55.7679,
48,Hobart,AU,83.05,scattered clouds,-42.8794,147.3294,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location']=f'{lat},{lng}'

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Mataura,NZ,75.47,broken clouds,-46.1927,168.8643,Ellie's Villa
12,Carnarvon,AU,81.54,clear sky,-24.8667,113.6333,Hospitality Carnarvon
13,Souillac,MU,79.36,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
16,Rikitea,PF,78.01,broken clouds,-23.1203,-134.9692,People ThankYou
28,Sao Filipe,CV,75.04,overcast clouds,14.8961,-24.4956,Hotel Xaguate


In [19]:
# 8a. Create the output File (CSV)
output_data_file='Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
max_temp=hotel_df['Max Temp']
fig=gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=max_temp,dissipating=False,
                              max_intensity=300,point_radius=4)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))